# Setting up Polytropic [TOV](https://en.wikipedia.org/wiki/Tolman%E2%80%93Oppenheimer%E2%80%93Volkoff_equation) Initial Data

## Authors: Phil Chang, Zachariah B. Etienne

## This module sets up initial data for a [TOV](https://en.wikipedia.org/wiki/Tolman%E2%80%93Oppenheimer%E2%80%93Volkoff_equation) star in *spherical, isotropic coordinates*

<font color='green'>**This module has been validated to exhibit convergence to zero of the Hamiltonian constraint violation at the expected order to the exact solution (see [start-to-finish TOV module](Tutorial-Start_to_Finish-BSSNCurvilinear-Setting_up_TOV_initial_data.ipynb) for full test). Note that convergence at the surface of the star is lower order due to the sharp drop to zero in $T^{\mu\nu}$.**</font>

### NRPy+ Source Code for this module: [TOV/TOV_Solver.py](../edit/TOV/TOV_Solver.py)

<a id='top'></a>$$\label{top}$$

## Introduction

[Tolman-Oppenheimer-Volkoff (TOV)](https://en.wikipedia.org/wiki/Tolman%E2%80%93Oppenheimer%E2%80%93Volkoff_equation) initial data are constructed from the solution of a system of [ODEs](https://en.wikipedia.org/wiki/Ordinary_differential_equation). The solution is spherically symmetric.

This module first constructs the system of ODEs and uses [NumPy](http://www.numpy.org/) to numerically solve the system and output the result to a file.

Then, a small C code is written to read in the file, interpolate the data to any desired radius, and construct the ADM spacetime quantities and corresponding $T^{\mu\nu}$.

## Table of Contents

1. [Step 1](#step1): Overview of the [Tolman-Oppenheimer-Volkoff (TOV)](https://en.wikipedia.org/wiki/Tolman%E2%80%93Oppenheimer%E2%80%93Volkoff_equation) solution to Einstein's equations
1. [Step 2](#step2): Python code for solving the TOV equations using NumPy's ODE solver, outputting the data to a file
    1. [Step 2.1](#step2p1): Validation of the TOV output file against a trusted version
1. [Step 3](#step3): C code for reading in data file and interpolating data to any desired radius
    1. [Step 3.1](#step3p1): Reading in the data file, generated in [Step 2](#step2).
    1. [Step 3.2](#step3p2): Interpolating the data to any desired radius
1. [Step 4](#step4): Construct ADM spacetime quantities and $T^{\mu\nu}$ from TOV solution
    1. [Step 4.1](#step4p1): Load NRPy+ modules and set basic parameters
    1. [Step 4.2](#step4p2): Constructing ADM spacetime quantities from the TOV data
    1. [Step 4.3](#step4p3): Constructing $T^{\mu\nu}$ from the TOV data
1. [Step 5](#step5): NRPy+ code validation: Confirm that ADM spacetime quantities and $T^{\mu\nu}$

<a id='step1'></a>

## Step 1: Overview of the [Tolman-Oppenheimer-Volkoff (TOV)](https://en.wikipedia.org/wiki/Tolman%E2%80%93Oppenheimer%E2%80%93Volkoff_equation) solution to Einstein's equations \[Back to [top](#top)\]
$$\label{step1}$$


The [TOV line element](https://en.wikipedia.org/wiki/Tolman%E2%80%93Oppenheimer%E2%80%93Volkoff_equation) in terms of the *Schwarzschild coordinate* $r$ is written (in the $-+++$ form):
$$
ds^2 = - c^2 e^\nu dt^2 + \left(1 - \frac{2Gm}{rc^2}\right)^{-1} dr^2 + r^2 d\Omega^2,
$$
where $m(r)$ is the mass-energy enclosed at a given $r$, and is equal to the total star's mass outside the stellar radius $r=R$.

In terms of the *isotropic coordinate* $\bar{r}$ with $G=c=1$ (i.e., the coordinate system and units we'd prefer to use), the ($-+++$ form) line element is written:
$$
ds^2 = - e^{\nu} dt^2 + e^{4\phi} \left(d\bar{r}^2 + \bar{r}^2 d\Omega^2\right),
$$
where $\phi$ here is the *conformal factor*.

Setting components of the above line element equal to one another, we get (in $G=c=1$ units):

\begin{align}
r^2 &= e^{4\phi} \bar{r}^2 \implies e^{4\phi} = \frac{r^2}{\bar{r}^2} \\
\left(1 - \frac{2m}{r}\right)^{-1} dr^2 &= e^{4\phi} d\bar{r}^2 \\
\implies \frac{d\bar{r}(r)}{dr} &= \left(1 - \frac{2m}{r} \right)^{-1/2} \frac{\bar{r}(r)}{r}.
\end{align}

The TOV equations provide radial ODEs for the pressure and $\nu$ (from [the Wikipedia article on the TOV solution](https://en.wikipedia.org/wiki/Tolman%E2%80%93Oppenheimer%E2%80%93Volkoff_equation)):

\begin{align}
\frac{dP}{dr} &= - \frac{1}{r} \left( \frac{\rho + P}{2} \right) \left(\frac{2 m}{r} + 8 \pi r^2 P\right) \left(1 - \frac{2 m}{r}\right)^{-1} \\
\frac{d \nu}{d r} &= \frac{1}{r}\left(1 - \frac{2 m}{r}\right)^{-1} \left(\frac{2 m}{r} + 8 \pi r^2 P\right) \\
\end{align}

Assuming a polytropic equation of state, which relates the pressure $P$ to the baryonic rest-mass density $\rho_B$,

$$
P(\rho_B) = K \rho_B^\Gamma,
$$
the specific internal energy will be given by
$$
\epsilon = \frac{P}{\rho_B (\Gamma - 1)},
$$

so the total mass-energy density $\rho$ is given by
$$
\rho = \rho_B (1 + \epsilon).
$$

Given this, the mass-energy $m(r)$ density is the solution to the ODE:
$$
\frac{m(r)}{dr} = 4\pi r^2 \rho(r)
$$

Thus the full set of ODEs that need to be solved is given by

\begin{align}
\frac{dP}{dr} &= - \frac{1}{r} \left( \frac{\rho + P}{2} \right) \left(\frac{2 m}{r} + 8 \pi r^2 P\right) \left(1 - \frac{2 m}{r}\right)^{-1} \\
\frac{d \nu}{d r} &= \frac{1}{r}\left(1 - \frac{2 m}{r}\right)^{-1} \left(\frac{2 m}{r} + 8 \pi r^2 P\right) \\
\frac{m(r)}{dr} &= 4\pi r^2 \rho(r) \\
\frac{d\bar{r}(r)}{dr} &= \left(1 - \frac{2m}{r} \right)^{-1/2} \frac{\bar{r}(r)}{r}.
\end{align}

<a id='step2'></a>

## Step 2: Python code for solving the TOV equations using NumPy's ODE solver, outputting the data to a file \[Back to [top](#top)\]

$$\label{step2}$$

The following code solves the above equations, and was largely written by **Phil Chang**.

In [1]:
## TOV SOLVER FOR SIMPLE POLYTROPES.
## Authors: Phil Chang, Zachariah B. Etienne

# Full documentation for this module may be found in the NRPy+ tutorial Jupyter notebook:
#  Tutorial-Start_to_Finish-BSSNCurvilinear-Setting_up_TOV_initial_data.ipynb

# Inputs:
# * Output data file name
# * rho_baryon_central, the central density of the TOV star.
# * n, the polytropic equation of state index. n=1 models cold, degenerate neutron star matter.
# * K_Polytrope, the polytropic constant.
# * Verbose output toggle (default = True)

# Output: An initial data file (default file name = "outputTOVpolytrope.txt") that well
#         samples the (spherically symmetric) solution both inside and outside the star.
#         It is up to the initial data module to perform the 1D interpolation to generate
#         the solution at arbitrary radius. The file has the following columns:
# Column 1: Schwarzschild radius
# Column 2: rho(r), *total* mass-energy density (as opposed to baryonic rest-mass density)
# Column 3: P(r), Pressure
# Column 4: m(r), mass enclosed
# Column 5: e^{nu(r)}, g_{tt}(r)
# Column 6: e^{4 phi(r)}, conformal factor g_{rr}(r)
# Column 7: rbar(r), Isotropic radius

# rbar refers to the isotropic radius, and
# R_Schw refers to the Schwarzschild radius

import numpy as np
import scipy.integrate as si
import math

n = 1.
rho_baryon_central = 0.129285

P0 = 1. # ZACH NOTES: CHANGED FROM 100.
gamma = 1. + 1./n
gam1 = gamma - 1.

def TOV_pressure( rho_baryon) : 
    return P0*rho_baryon**gamma

def TOV_rhs(r_Schw, y) : 
# In \tilde units
#
    P    = y[0]
    m    = y[1]
    nu   = y[2]
    rbar = y[3]
    
    dPdrSchw    = 0.
    drbardrSchw = 0.
    
    rho_baryon = (P/P0)**(1./gamma)
    rho = rho_baryon + P/gam1 # rho is the *total* mass-energy density!
    if( r_Schw < 1e-4 or m <= 0.): 
        m = 4*math.pi/3. * rho*r_Schw**3
        dPdrSchw = -(rho + P)*(4.*math.pi/3.*r_Schw*rho + 4.*math.pi*r_Schw*P)/(1.-8.*math.pi*rho*r_Schw*r_Schw)
        drbardrSchw = 1./(1. - 8.*math.pi*rho*r_Schw*r_Schw)**0.5
    else:
        dPdrSchw = -(rho + P)*(m + 4.*math.pi*r_Schw**3*P)/(r_Schw*r_Schw*(1.-2.*m/r_Schw))
        drbardrSchw = 1./(1. - 2.*m/r_Schw)**0.5*rbar/r_Schw
    
    dmdrSchw  =  4.*math.pi*r_Schw*r_Schw*rho
    dnudrSchw = -2./(P + rho)*dPdrSchw
    return [dPdrSchw, dmdrSchw, dnudrSchw, drbardrSchw]

def integrateStar( P, dumpData = False):
    integrator = si.ode(TOV_rhs).set_integrator('dop853')
    y0 = [P, 0., 0., 0.]
    integrator.set_initial_value(y0,0.)
    dr_Schw = 1e-5
    P = y0[0]

    PArr      = []
    r_SchwArr = []
    mArr      = []
    nuArr     = []
    rbarArr   = []

    r_Schw = 0.

    while integrator.successful() and P > 1e-9*y0[0] : 
        P, m, nu, rbar = integrator.integrate(r_Schw + dr_Schw)
        r_Schw = integrator.t

        dPdrSchw, dmdrSchw, dnudrSchw, drbardrSchw = TOV_rhs( r_Schw+dr_Schw, [P,m,nu,rbar])
        dr_Schw = 0.1*min(abs(P/dPdrSchw), abs(m/dmdrSchw))
        dr_Schw = min(dr_Schw, 1e-2)
        PArr.append(P)
        r_SchwArr.append(r_Schw)
        mArr.append(m)
        nuArr.append(nu)
        rbarArr.append(rbar)

    M = mArr[-1]
    R_Schw = r_SchwArr[-1]

    # Apply integration constant to ensure rbar is continuous across TOV surface
    for ii in range(len(rbarArr)):
        rbarArr[ii] *= 0.5*(np.sqrt(R_Schw*(R_Schw - 2.0*M)) + R_Schw - M) / rbarArr[-1]
    
    nuArr_np = np.array(nuArr)
    # Rescale solution to nu so that it satisfies BC: exp(nu(R))=exp(nutilde-nu(r=R)) * (1 - 2m(R)/R)
    #   Thus, nu(R) = (nutilde - nu(r=R)) + log(1 - 2*m(R)/R)
    nuArr_np = nuArr_np - nuArr_np[-1] + math.log(1.-2.*mArr[-1]/r_SchwArr[-1])

    r_SchwArrExtend_np = 10.**(np.arange(0.01,5.0,0.01))*r_SchwArr[-1]
    
    r_SchwArr.extend(r_SchwArrExtend_np)
    mArr.extend(r_SchwArrExtend_np*0. + M)
    PArr.extend(r_SchwArrExtend_np*0.)
    exp2phiArr_np = np.append( np.exp(nuArr_np), 1. - 2.*M/r_SchwArrExtend_np)
    nuArr.extend(np.log(1. - 2.*M/r_SchwArrExtend_np))
    rbarArr.extend( 0.5*(np.sqrt(r_SchwArrExtend_np**2 - 2.*M*r_SchwArrExtend_np) + r_SchwArrExtend_np - M) )

    # Appending to a Python array does what one would reasonably expect.
    #   Appending to a numpy array allocates space for a new array with size+1,
    #   then copies the data over... over and over... super inefficient.
    r_SchwArr_np          = np.array(r_SchwArr)
    PArr_np               = np.array(PArr)
    rho_baryonArr_np      = (PArr_np/P0)**(1./gamma)
    mArr_np               = np.array(mArr)
    rbarArr_np            = np.array(rbarArr)
    confFactor_exp4phi_np = (r_SchwArr_np/rbarArr_np)**2

    # Compute the *total* mass-energy density (as opposed to the *baryonic* mass density)
    rhoArr_np = []
    for i in range(len(rho_baryonArr_np)):
        rhoArr_np.append(rho_baryonArr_np[i] + PArr_np[i]/(gamma - 1.))
    
    print(len(r_SchwArr_np),len(rhoArr_np),len(PArr_np),len(mArr_np),len(exp2phiArr_np))
    np.savetxt("outputTOVpolytrope.txt", zip(r_SchwArr_np,rhoArr_np,PArr_np,mArr_np,exp2phiArr_np,confFactor_exp4phi_np,rbarArr_np), 
               fmt="%.15e")

    return R_Schw, M

R_Schw_TOV,M_TOV = integrateStar(TOV_pressure(rho_baryon_central), True)
print("Just generated a TOV star with R_Schw = "+str(R_Schw_TOV)+" , M = "+str(M_TOV)+" , M/R_Schw = "+str(M_TOV/R_Schw_TOV)+" .")

(1051, 1051, 1051, 1051, 1051)
Just generated a TOV star with R_Schw = 0.956568142523 , M = 0.14050303285288188 , M/R_Schw = 0.1468824086931645 .


<a id='step2p1'></a>

## Step 2.1: Validation of the TOV output file against a trusted version
$$\label{step2p1}$$
\[Back to [top](#top)\]

Here, as a code validation check, we verify agreement in the SymPy expressions for these TOV initial data between

1. this tutorial and 
2. the NRPy+ [TOV.TOV_Solver](../edit/TOV/TOV_Solver.py) module.

In [2]:
import filecmp

import TOV.TOV_Solver as TOV
TOV.TOV_Solver("outputTOVpolytrope-validation.txt",rho_baryon_central=0.129285, n_Polytrope=1.0, K_Polytrope=1.0,
               verbose = False)

if filecmp.cmp('outputTOVpolytrope.txt',
               'outputTOVpolytrope-validation.txt') == False:
    print("ERROR: TOV initial data test FAILED!")
    exit(1)
else:
    print("TOV initial data test PASSED.")

TOV initial data test PASSED.


<a id='step3'></a>

## Step 3: C code for reading TOV star initial data and interpolating data to numerical grids \[Back to [top](#top)\]

$$\label{step3}$$

<a id='step3p1'></a>

### Step 3.1: C code for reading TOV star initial data file \[Back to [top](#top)\]
$$\label{step3p1}$$


In [3]:
%%writefile TOV/read_datafile__set_arrays.h

int read_datafile__set_arrays(FILE *in1Dpolytrope, REAL *r_Schw_arr,REAL *rho_arr,REAL *P_arr,REAL *M_arr,REAL *expnu_arr,REAL *exp4phi_arr,REAL *rbar_arr) {
  char * line = NULL;

  size_t len = 0;
  ssize_t read;

  int which_line = 0;
  while ((read = getline(&line, &len, in1Dpolytrope)) != -1) {
    // Define the line delimiters (i.e., the stuff that goes between the data on a given
    //     line of data.  Here, we define both spaces " " and tabs "\t" as data delimiters.
    const char delimiters[] = " \t";

    //Now we define "token", a pointer to the first column of data
    char *token;

    //Each successive time we call strtok(NULL,blah), we read in a new column of data from
    //     the originally defined character array, as pointed to by token.

    token=strtok(line, delimiters); if(token==NULL) { printf("BADDDD\n"); return 1; }
    r_Schw_arr[which_line]  = strtod(token, NULL); token = strtok( NULL, delimiters );
    rho_arr[which_line]     = strtod(token, NULL); token = strtok( NULL, delimiters );
    P_arr[which_line]       = strtod(token, NULL); token = strtok( NULL, delimiters );
    M_arr[which_line]       = strtod(token, NULL); token = strtok( NULL, delimiters );
    expnu_arr[which_line]   = strtod(token, NULL); token = strtok( NULL, delimiters );
    exp4phi_arr[which_line] = strtod(token, NULL); token = strtok( NULL, delimiters );
    rbar_arr[which_line]    = strtod(token, NULL);

    which_line++;
  }
  free(line);
  return 0;
}

Overwriting TOV/read_datafile__set_arrays.h


<a id='step3p2'></a>

### Step 3.2: C code for interpolating TOV data to desired radial coordinate $\bar{r}$ \[Back to [top](#top)\]
$$\label{step3p2}$$

The TOV data file written in [Step 2](#step2) above stored $\left(r,\rho(r),P(r),M(r),e^{\nu(r),\bar{r}}\right)$, where $\rho(r)$ is the total mass-energy density (cf. $\rho_{\text{baryonic}}$) at a fixed number of radii. This C code implements [Lagrange polynomial interpolation](https://en.wikipedia.org/wiki/Polynomial_interpolation) to interpolate $\left(r,\rho(r),P(r),M(r),e^{\nu(r)}\right)$ to any given input radius $\bar{r}$.

In [4]:
%%writefile TOV/TOV_interpolate_1D.h

void TOV_interpolate_1D(REAL rrbar,const REAL Rbar,const int Rbar_idx,const int interp_stencil_size,
                        const int numlines_in_file,const REAL *r_Schw_arr,const REAL *rho_arr,const REAL *P_arr,const REAL *M_arr,const REAL *expnu_arr,const REAL *exp4phi_arr,const REAL *rbar_arr,
                        REAL *rho,REAL *P,REAL *M,REAL *expnu,REAL *exp4phi) {
  // Find interpolation index using Bisection root-finding algorithm:
  int bisection_idx_finder(const REAL rr, const int numlines_in_file, const REAL *rbar_arr) {
    int x1 = 0;
    int x2 = numlines_in_file-1;
    REAL y1 = rrbar-rbar_arr[x1];
    REAL y2 = rrbar-rbar_arr[x2];
    if(y1*y2 >= 0) {
      fprintf(stderr,"INTERPOLATION BRACKETING ERROR %e | %e %e\n",rr,y1,y2);
      exit(1);
    }
    for(int i=0;i<numlines_in_file;i++) {
      int x_midpoint = (x1+x2)/2;
      REAL y_midpoint = rrbar-rbar_arr[x_midpoint];
      if(y_midpoint*y1 < 0) {
        x2 = x_midpoint;
        y2 = y_midpoint;
      } else {
        x1 = x_midpoint;
        y1 = y_midpoint;
      }
      if( abs(x2-x1) == 1 ) {
        // If rbar_arr[x1] is closer to rrbar than rbar_arr[x2] then return x1:
        if(fabs(rrbar-rbar_arr[x1]) < fabs(rrbar-rbar_arr[x2])) return x1;
        // Otherwiser return x2:
        return x2;
      }
    }
    fprintf(stderr,"INTERPOLATION BRACKETING ERROR: DID NOT CONVERGE.\n");
    exit(1);
  }

  // For this case, we know that for all functions, f(r) = f(-r)
  if(rrbar < 0) rrbar = -rrbar;

  // First find the central interpolation stencil index:
  int idx = bisection_idx_finder(rrbar,numlines_in_file,rbar_arr);
  
#ifdef MAX
#undef MAX
#endif
#define MAX(A, B) ( ((A) > (B)) ? (A) : (B) )

  int idxmin = MAX(0,idx-interp_stencil_size/2-1);

#ifdef MIN
#undef MIN
#endif
#define MIN(A, B) ( ((A) < (B)) ? (A) : (B) )

  // -= Do not allow the interpolation stencil to cross the star's surface =-
  // max index is when idxmin + (interp_stencil_size-1) = Rbar_idx
  //  -> idxmin at most can be Rbar_idx - interp_stencil_size + 1
  if(rrbar < Rbar) {
    idxmin = MIN(idxmin,Rbar_idx - interp_stencil_size + 1);
  } else {
    idxmin = MAX(idxmin,Rbar_idx+1);
    idxmin = MIN(idxmin,numlines_in_file - interp_stencil_size + 1);
  }

  // Now perform Lagrange polynomial interpolation:

  // First set the interpolation coefficients:
  REAL rbar_sample[interp_stencil_size];
  for(int i=idxmin;i<idxmin+interp_stencil_size;i++) {
    rbar_sample[i-idxmin] = rbar_arr[i];
  }
  REAL l_i_of_r[interp_stencil_size];
  for(int i=0;i<interp_stencil_size;i++) {
    REAL numer = 1.0;
    REAL denom = 1.0;
    for(int j=0;j<i;j++) {
      numer *= rrbar - rbar_sample[j];
      denom *= rbar_sample[i] - rbar_sample[j];
    }
    for(int j=i+1;j<interp_stencil_size;j++) {
      numer *= rrbar - rbar_sample[j];
      denom *= rbar_sample[i] - rbar_sample[j];
    }
    l_i_of_r[i] = numer/denom;
  }

  // Then perform the interpolation:
  *rho = 0.0;
  *P = 0.0;
  *M = 0.0;
  *expnu = 0.0;
  *exp4phi = 0.0;

  REAL r_Schw = 0.0;
  for(int i=idxmin;i<idxmin+interp_stencil_size;i++) {
    r_Schw   += l_i_of_r[i-idxmin] * r_Schw_arr[i];
    *rho     += l_i_of_r[i-idxmin] * rho_arr[i];
    *P       += l_i_of_r[i-idxmin] * P_arr[i];
    *M       += l_i_of_r[i-idxmin] * M_arr[i];
    *expnu   += l_i_of_r[i-idxmin] * expnu_arr[i];
    *exp4phi += l_i_of_r[i-idxmin] * exp4phi_arr[i];
  }

  if(rrbar > Rbar) {
    *rho   = 0;
    *P     = 0;
    *M     = M_arr[Rbar_idx+1];
    *expnu = 1. - 2.*(*M) / r_Schw;
    *exp4phi = pow(r_Schw / rrbar,2.0);
  }
}

Overwriting TOV/TOV_interpolate_1D.h


<a id='step4'></a>

## Step 4: Construct ADM spacetime quantities and $T^{\mu\nu}$  from TOV solution \[Back to [top](#top)\]
$$\label{step4}$$

<a id='step4p1'></a>

## Step 4.1: Load NRPy+ modules and set basic parameters \[Back to [top](#top)\]
$$\label{step4p1}$$

In [5]:
# Step 4.1: Load NRPy+ modules and set basic parameters

import sympy as sp
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
from outputC import *
import reference_metric as rfm

# All gridfunctions will be written in terms of spherical coordinates (r, th, ph):
r,th,ph = sp.symbols('r th ph', real=True)

thismodule = "TOV"

DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Input parameters read in from the TOV data file:
rbar,expnu,exp4phi,P,rho = par.Cparameters("REAL", thismodule, ["rbar","expnu","exp4phi","P","rho"])

<a id='step4p2'></a>

## Step 4.2: Constructing ADM spacetime quantities from the TOV data \[Back to [top](#top)\]

$$\label{step4p2}$$

The [TOV line element](https://en.wikipedia.org/wiki/Tolman%E2%80%93Oppenheimer%E2%80%93Volkoff_equation) in *Schwarzschild coordinates* is written (in the $-+++$ form):
$$
ds^2 = - c^2 e^\nu dt^2 + \left(1 - \frac{2GM}{rc^2}\right)^{-1} dr^2 + r^2 d\Omega^2.
$$

In *isotropic coordinates* with $G=c=1$ (i.e., the coordinate system and units we'd prefer to use), the ($-+++$ form) line element is written:
$$
ds^2 = - e^{\nu} dt^2 + e^{4\phi} \left(d\bar{r}^2 + \bar{r}^2 d\Omega^2\right),
$$
where $\phi$ here is the *conformal factor*.

The ADM 3+1 line element for this diagonal metric in isotropic spherical coordinates is given by:
$$
ds^2 = (-\alpha^2 + \beta_k \beta^k) dt^2 + \gamma_{\bar{r}\bar{r}} d\bar{r}^2 + \gamma_{\theta\theta} d\theta^2+ \gamma_{\phi\phi} d\phi^2,
$$

from which we can immediately read off the ADM quantities:
\begin{align}
\gamma_{\bar{r}\bar{r}} &= e^{4\phi}\\
\gamma_{\theta\theta} &= e^{4\phi} \bar{r}^2 \\
\gamma_{\phi\phi} &= e^{4\phi} \bar{r}^2 \sin^2 \theta \\
K_{ij} &= 0 \\
\alpha &= e^{\nu(\bar{r})/2} \\
\beta^i = B^i &= 0 \\
\end{align}

In [6]:
# Step 4.2: Construct ADM quantities:

# *** The physical spatial metric in spherical basis ***
# In isotropic coordinates, 
#  gamma_{ij} = e^{4 phi} eta_{ij}, 
# where eta is the flat-space 3-metric in spherical coordinates
gammaSphDD = ixp.zerorank2()
gammaSphDD[0][0] = exp4phi
gammaSphDD[1][1] = exp4phi * rbar**2
gammaSphDD[2][2] = exp4phi * rbar**2*sp.sin(th)**2

# *** The extrinsic curvature in spherical basis ***
# K_{ij} = 0 for the TOV solution
KSphDD = ixp.zerorank2()

# *** The lapse and shift in spherical basis ***
# alpha = exp^{nu/2} for the TOV solution
# \beta^i = 0 for the TOV solution
alphaSph = sp.sqrt(expnu)
betaSphU = ixp.zerorank1()
BSphSphU = ixp.zerorank1()

<a id='step4p3'></a>

## Step 4.3: Constructing $T^{\mu\nu}$ from the TOV data \[Back to [top](#top)\]
$$\label{step4p3}$$

We will also need the stress-energy tensor $T^{\mu\nu}$. [As discussed here](https://en.wikipedia.org/wiki/Tolman%E2%80%93Oppenheimer%E2%80%93Volkoff_equation), the stress-energy tensor is diagonal:

\begin{align}
T^t_t &= -\rho \\
T^i_j &= P \delta^i_j \\
\text{All other components of }T^\mu_\nu &= 0.
\end{align}

Since $\beta^i=0$ the inverse metric expression simplifies to (Eq. 4.49 in [Gourgoulhon](https://arxiv.org/pdf/gr-qc/0703035.pdf)):
$$
g^{\mu\nu} = \begin{pmatrix} 
-\frac{1}{\alpha^2} & \frac{\beta^i}{\alpha^2} \\
\frac{\beta^i}{\alpha^2} & \gamma^{ij} - \frac{\beta^i\beta^j}{\alpha^2}
\end{pmatrix} =
\begin{pmatrix} 
-\frac{1}{\alpha^2} & 0 \\
0 & \gamma^{ij}
\end{pmatrix},
$$

and since the 4-metric is diagonal we get

\begin{align}
\gamma^{\bar{r}\bar{r}} &= e^{-4\phi}\\
\gamma^{\theta\theta} &= e^{-4\phi}\frac{1}{\bar{r}^2} \\
\gamma^{\phi\phi} &= e^{-4\phi}\frac{1}{\bar{r}^2 \sin^2 \theta}.
\end{align}

Thus raising $T^\mu_\nu$ yields a diagonal $T^{\mu\nu}$

\begin{align}
T^{tt} &= -g^{tt} \rho = \frac{1}{\alpha^2} \rho = e^{-\nu(\bar{r})} \rho \\
T^{\bar{r}\bar{r}} &= g^{\bar{r}\bar{r}} P = \frac{1}{e^{4 \phi}} P \\
T^{\theta\theta} &= g^{\theta\theta} P = \frac{1}{e^{4 \phi}\bar{r}^2} P\\
T^{\phi\phi} &= g^{\phi\phi} P = \frac{1}{e^{4\phi}\bar{r}^2 \sin^2 \theta} P 
\end{align}

In [7]:
# Step 4.3: Construct T^{mu nu}:
T4UU = ixp.zerorank2(DIM=4)

# T^tt = e^(-nu) * rho
T4UU[0][0] = rho / expnu
# T^{ii} = P / gamma_{ii}
for i in range(3):
    T4UU[i+1][i+1] = P / gammaSphDD[i][i]

<a id='step5'></a>

## Step 5: NRPy+ Module Code Validation \[Back to [top](#top)\]
$$\label{step5}$$

Here, as a code validation check, we verify agreement in the SymPy expressions for TOV initial data between

1. this tutorial and 
2. the NRPy+ [TOV.TOV_ADM_T4UUmunu](../edit/TOV/TOV_ADM_T4UUmunu.py) module.

As all input quantities are functions of $r$, we will simply read the solution from file and interpolate it to the values of $r$ needed by the initial data.

1. First we define functions ID_TOV_ADM_quantities() and ID_TOV_TUPMUNU() that call the [1D TOV interpolator function](../edit/TOV/tov_interp.h) to evaluate the ADM spacetime quantities and $T^{\mu\nu}$, respectively, at any given point $(r,\theta,\phi)$ in the Spherical basis. All quantities are defined as above.
1. Next we will construct the BSSN/ADM source terms $\{S_{ij},S_{i},S,\rho\}$ in the Spherical basis
1. Then we will perform the Jacobian transformation on $\{S_{ij},S_{i},S,\rho\}$ to the desired (xx0,xx1,xx2) basis
1. Next we call the *Numerical* Spherical ADM$\to$Curvilinear BSSN converter function to conver the above ADM quantities to the rescaled BSSN quantities in the desired curvilinear coordinate system: [BSSN/ADM_Numerical_Spherical_or_Cartesian_to_BSSNCurvilinear.py](../edit/BSSN/ADM_Numerical_Spherical_or_Cartesian_to_BSSNCurvilinear.py); [\[**tutorial**\]](Tutorial-ADM_Initial_Data-Converting_Numerical_ADM_Spherical_or_Cartesian_to_BSSNCurvilinear.ipynb).



### Output to $\LaTeX$, then PDF

In [8]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-ADM_and_T4UUmunu_Initial_Data-TOV.ipynb
!pdflatex -interaction=batchmode Tutorial-ADM_and_T4UUmunu_Initial_Data-TOV.tex
!pdflatex -interaction=batchmode Tutorial-ADM_and_T4UUmunu_Initial_Data-TOV.tex
!pdflatex -interaction=batchmode Tutorial-ADM_and_T4UUmunu_Initial_Data-TOV.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-ADM_and_T4UUmunu_Initial_Data-TOV.ipynb to latex
[NbConvertApp] Writing 63977 bytes to Tutorial-ADM_and_T4UUmunu_Initial_Data-TOV.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
